In [1]:
import os
os.chdir("../")
os.getcwd()

'/home/izam/coding/TradePilot'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PriceDataIngestionConfig:
    root_dir: Path
    data_dir: str
    companies: dict
    date_start: str
    date_end: str
    period: str

In [3]:
from TradePilot.constants import *
from TradePilot.utils.common import create_directories, read_yaml

In [4]:

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        price_data_config_filepath = PRICE_DATA_CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH,
        ):

        self.config = read_yaml(config_filepath)
        self.data_config = read_yaml(price_data_config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_price_data_ingestion_config(self) -> PriceDataIngestionConfig:
        config = self.config.price_ingestion
        companies = self.data_config.Companies
        data_properties = self.data_config.DataFrame

        create_directories([config.root_dir])

        price_data_ingestion_config = PriceDataIngestionConfig(
            root_dir=config.root_dir,
            data_dir=config.data_dir,
            companies=companies,
            date_start=data_properties.start,
            date_end=data_properties.end,
            period=data_properties.period,
        )

        return price_data_ingestion_config

In [5]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date
import numpy as np

In [6]:
from TradePilot import logger
from TradePilot.utils.common import create_directories, save_json

In [7]:
# component

class PriceDataIngestion:
    def __init__(self, config: PriceDataIngestionConfig) -> None:
        self.config = config

    def download_data(self):
        data_dir = os.path.join(self.config.root_dir, self.config.data_dir)
        create_directories([data_dir])

        companies = self.config.companies
        start_date = self.config.date_start
        end_date = self.config.date_end
        period = self.config.period
        df_rows = {}

        for company, ticker in companies.items():
            data = yf.download(tickers=ticker, start=start_date, end=end_date, period=period)
            df_rows[company] = data.shape[0]
            
            if df_rows[company]:
                logger.info(f"downloaded the data of the company {company} with {df_rows[company]} data points")
            else:
                logger.info(f"failed to fetch the data of the company {company}")

            file_name = os.path.join(data_dir, f"{company}.csv")
            data.to_csv(file_name)
            
            logger.info(f"saved the file {file_name}")

        json_file_path = os.path.join(self.config.root_dir, "data_info.json")
        save_json(path=json_file_path, data=df_rows)

In [8]:
try:
    config = ConfigurationManager()
    price_data_ingestion_config = config.get_price_data_ingestion_config()
    price_data_ingestion = PriceDataIngestion(config=price_data_ingestion_config)
    price_data_ingestion.download_data()
except Exception as e:
    raise e

[2024-01-06 19:36:24,630: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-01-06 19:36:24,633: INFO: common: yaml file: config/price_data_config.yaml loaded successfully]
[2024-01-06 19:36:24,633: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-06 19:36:24,634: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-01-06 19:36:24,635: INFO: common: created directory at: artifacts]
[2024-01-06 19:36:24,635: INFO: common: created directory at: artifacts/data_ingestion]
[2024-01-06 19:36:24,636: INFO: common: created directory at: artifacts/data_ingestion/stock_prices]
[*********************100%%**********************]  1 of 1 completed
[2024-01-06 19:36:25,141: INFO: 2386939323: downloaded the data of the company Reliance with 6281 data points]
[2024-01-06 19:36:25,175: INFO: 2386939323: saved the file artifacts/data_ingestion/stock_prices/Reliance.csv]
[2024-01-06 19:36:25,175: INFO: common: json file saved at: artifacts/data_ingestion/d